
# 데이터 불러오기

In [ ]:
import matplotlib as plt
plt.rc('axes', unicode_minus=False) # 마이너스 기호 깨짐 방지
# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic') # 폰트 적용

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/MyDrive/파이썬/00_mimi_project/"

In [ ]:
train_trans = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 구매기록 학습데이터
test_trans = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 구매기록 테스트데이터

store_train = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 고객 ID 및 정답데이터
store_submission = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 테스트용 고객 ID 데이터(제출파일 양식)


train_trans.shape , test_trans.shape, store_train.shape, store_submission.shape

((523105, 7), (441196, 7), (14940, 2), (12225, 2))

In [ ]:
train_trans["구매일시"] = pd.to_datetime(train_trans["구매일시"])
test_trans["구매일시"] = pd.to_datetime(test_trans["구매일시"])

In [ ]:
train_ft = store_train.copy()
test_ft = store_submission.iloc[:,:1].copy()

# ==================================================

# EDA

# Feature engineering

## 특성추가

In [ ]:
agg_list = [
        ('내점일수',lambda x : x.dt.date.nunique() ),
        ('구매주기',lambda x : ( x.max() - x.min() ).days / x.dt.date.nunique() ),
        ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4)),
        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.dayofweek.value_counts().index[0]),
        ('거래개월수', lambda x: x.dt.month.nunique() ),
    ]

tmp = train_trans.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,target,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수
0,train_0,1.0,11,23.090909,0.250000,0.050000,0.250000,0.400000,0.300000,3,7
1,train_1,1.0,21,16.714286,0.023810,0.357143,0.166667,0.357143,0.119048,3,11
2,train_2,0.0,56,6.339286,0.210526,0.464912,0.140351,0.175439,0.219298,0,12
3,train_3,0.0,92,3.728261,0.189573,0.379147,0.180095,0.236967,0.203791,3,12
4,train_4,0.0,27,11.555556,0.258065,0.112903,0.612903,0.209677,0.064516,4,10


In [ ]:
tmp = test_trans.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수
0,test_0,5,32.600000,0.571429,0.285714,0.285714,0.428571,0.000000,6,4
1,test_1,3,54.666667,0.250000,0.000000,0.000000,0.750000,0.250000,0,3
2,test_2,2,8.000000,0.000000,0.000000,0.600000,0.400000,0.000000,0,2
3,test_3,16,16.937500,0.031250,0.062500,0.468750,0.343750,0.125000,3,7
4,test_4,9,29.444444,0.476190,0.380952,0.285714,0.238095,0.095238,6,4


In [ ]:
agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
    ]

tmp = train_trans.groupby('ID')["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,방문지점수,주구매지점
0,test_0,5,32.600000,0.571429,0.285714,0.285714,0.428571,0.000000,6,4,1,A202000
1,test_1,3,54.666667,0.250000,0.000000,0.000000,0.750000,0.250000,0,3,2,A373000
2,test_2,2,8.000000,0.000000,0.000000,0.600000,0.400000,0.000000,0,2,1,A144000
3,test_3,16,16.937500,0.031250,0.062500,0.468750,0.343750,0.125000,3,7,1,A112000
4,test_4,9,29.444444,0.476190,0.380952,0.285714,0.238095,0.095238,6,4,3,A112000


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(handle_unknown='ignore')
col=['주구매지점']

coder=encoder.fit_transform(train_ft[col])
train_ft=pd.concat([train_ft,pd.DataFrame(coder.toarray(),columns=encoder.get_feature_names_out(col))],axis=1)
train_ft.drop('주구매지점',axis=1,inplace=True)
train_ft

coder=encoder.fit_transform(test_ft[col])
test_ft=pd.concat([test_ft,pd.DataFrame(coder.toarray(),columns=encoder.get_feature_names_out(col))],axis=1)
test_ft.drop('주구매지점',axis=1,inplace=True)
test_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,방문지점수,주구매지점_A112000,주구매지점_A144000,주구매지점_A202000,주구매지점_A373000
0,test_0,5,32.600000,0.571429,0.285714,0.285714,0.428571,0.000000,6,4,1,0.0,0.0,1.0,0.0
1,test_1,3,54.666667,0.250000,0.000000,0.000000,0.750000,0.250000,0,3,2,0.0,0.0,0.0,1.0
2,test_2,2,8.000000,0.000000,0.000000,0.600000,0.400000,0.000000,0,2,1,0.0,1.0,0.0,0.0
3,test_3,16,16.937500,0.031250,0.062500,0.468750,0.343750,0.125000,3,7,1,1.0,0.0,0.0,0.0
4,test_4,9,29.444444,0.476190,0.380952,0.285714,0.238095,0.095238,6,4,3,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_12220,6,47.833333,0.000000,0.166667,0.166667,0.250000,0.416667,3,5,3,1.0,0.0,0.0,0.0
12221,test_12221,14,24.785714,0.238095,0.380952,0.000000,0.238095,0.380952,1,8,2,0.0,1.0,0.0,0.0
12222,test_12222,28,12.464286,0.315789,0.157895,0.263158,0.289474,0.289474,2,10,3,0.0,0.0,0.0,1.0
12223,test_12223,115,3.008696,0.202335,0.171206,0.311284,0.287938,0.229572,1,12,2,0.0,0.0,1.0,0.0


In [ ]:
tmp = train_trans.groupby('ID')['지점코드'].apply(lambda x : np.mean(x == 'A144000')).reset_index().rename(columns = {'지점코드':'A144000_방문율'})
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')['지점코드'].apply(lambda x : np.mean(x == 'A144000')).reset_index().rename(columns = {'지점코드':'A144000_방문율'})
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,방문지점수,주구매지점_A112000,주구매지점_A144000,주구매지점_A202000,주구매지점_A373000,A144000_방문율
0,test_0,5,32.600000,0.571429,0.285714,0.285714,0.428571,0.000000,6,4,1,0.0,0.0,1.0,0.0,0.000000
1,test_1,3,54.666667,0.250000,0.000000,0.000000,0.750000,0.250000,0,3,2,0.0,0.0,0.0,1.0,0.000000
2,test_2,2,8.000000,0.000000,0.000000,0.600000,0.400000,0.000000,0,2,1,0.0,1.0,0.0,0.0,1.000000
3,test_3,16,16.937500,0.031250,0.062500,0.468750,0.343750,0.125000,3,7,1,1.0,0.0,0.0,0.0,0.000000
4,test_4,9,29.444444,0.476190,0.380952,0.285714,0.238095,0.095238,6,4,3,1.0,0.0,0.0,0.0,0.238095


In [ ]:
agg_list = [
             ('브랜드코드_nunique', 'nunique'),
             ]

tmp = train_trans.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,방문지점수,주구매지점_A112000,주구매지점_A144000,주구매지점_A202000,주구매지점_A373000,A144000_방문율,브랜드코드_nunique
0,test_0,5,32.600000,0.571429,0.285714,0.285714,0.428571,0.000000,6,4,1,0.0,0.0,1.0,0.0,0.000000,5
1,test_1,3,54.666667,0.250000,0.000000,0.000000,0.750000,0.250000,0,3,2,0.0,0.0,0.0,1.0,0.000000,4
2,test_2,2,8.000000,0.000000,0.000000,0.600000,0.400000,0.000000,0,2,1,0.0,1.0,0.0,0.0,1.000000,5
3,test_3,16,16.937500,0.031250,0.062500,0.468750,0.343750,0.125000,3,7,1,1.0,0.0,0.0,0.0,0.000000,19
4,test_4,9,29.444444,0.476190,0.380952,0.285714,0.238095,0.095238,6,4,3,1.0,0.0,0.0,0.0,0.238095,13


In [ ]:
agg_list = [
            ('중분류_nunique', 'nunique'),
            # ('주구매_중분류', lambda x: x.mode()[0]), 추가 x
            ]

tmp = train_trans.groupby('ID')["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')

In [ ]:
middle_keywords=['GBR  지원','TV.VTR','TV_VTR','가스렌지','가전특정','건강용품', '건식품', '건어물', '곡물', '골프(LC)', '골프(NB)', '골프(국내)', '골프(단품)', '골프(수입)', '골프(용품)', '골프단품', '골프웨어', '과자', '교복행사', '구두수선', '국내부띠끄', '기타식품', '내셔날', '내셔널', '냉난방', '냉동식품', '냉장고_세탁기', '냉장식품', '니트', '단품(트래디셔널)', '단품행사', '디자이너니트',
'디자이너부띠끄', '디자이너숍', '디자이너캐릭터', '라디오.카세트', '라이센스', '레이디숍A', '레포츠',
'레포츠단품', '로얄부틱2F', '로얄수입행사', '마춤', '면류', '모피.피혁', '모피_피혁', '문구', '문화', '부띠끄', '부띠끄행사', '상품개발지원', '생선', '세탁기_냉장고', '소파', '소형가구(가구)', '쇼파', '수예',
 '수예침장','수예행사','수입_종합가구','수입구두','수입도자기','수입부띠끄','수입슈즈','수입캐주얼','수입피혁','슈즈','스포츠단품',
 '아동','아동단품',
 '아동복','아동잡화',
 '아동특선', '야채',
 '엘레강스','엘레강스부틱','영잡화',
 '완구','용기보증',
 '용품','원목(주니어)','유아','음료','의류기타',
 '인스탄트식품','임대골프', '잡화토탈',
 '전문가구(가구)',
'전화기',
'정육',
'조명',
 '조미료','종합_수입','주방용품',
 '지갑_벨트','직수입침구','청과','침구','침대',
 '카세트_전화기', '카페트',
 '카페트_대자리',
'캐쥬얼단품',
'크리스탈',
'타운','타운단품',
 '통병조림',
'통판',
 '트래디셔널캐쥬얼','특선행사','특판','페레  지원','피혁A행사','해외SHOP','행사_단품','행사구두(5F)','훼미닌부틱']

In [ ]:
tmp = train_trans.groupby('ID')['중분류'].agg(lambda x : np.mean(x.isin(middle_keywords))).reset_index().rename(columns = {'중분류':'중분류_구매_비율'})
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')['중분류'].agg(lambda x : np.mean(x.isin(middle_keywords))).reset_index().rename(columns = {'중분류':'중분류_구매_비율'})
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,방문지점수,주구매지점_A112000,주구매지점_A144000,주구매지점_A202000,주구매지점_A373000,A144000_방문율,브랜드코드_nunique,중분류_nunique,중분류_구매_비율
0,test_0,5,32.600000,0.571429,0.285714,0.285714,0.428571,0.000000,6,4,1,0.0,0.0,1.0,0.0,0.000000,5,4,0.857143
1,test_1,3,54.666667,0.250000,0.000000,0.000000,0.750000,0.250000,0,3,2,0.0,0.0,0.0,1.0,0.000000,4,4,0.000000
2,test_2,2,8.000000,0.000000,0.000000,0.600000,0.400000,0.000000,0,2,1,0.0,1.0,0.0,0.0,1.000000,5,4,0.000000
3,test_3,16,16.937500,0.031250,0.062500,0.468750,0.343750,0.125000,3,7,1,1.0,0.0,0.0,0.0,0.000000,19,12,0.000000
4,test_4,9,29.444444,0.476190,0.380952,0.285714,0.238095,0.095238,6,4,3,1.0,0.0,0.0,0.0,0.238095,13,12,0.095238


In [ ]:
tmp = (train_trans[train_trans['중분류'].isin(middle_keywords)].groupby('ID')['구매가격'].agg('sum')).reset_index().rename(columns = {'구매가격':'중분류_주요물품_구매액_합계'})
train_ft = train_ft.merge(tmp,how='left',on = 'ID')

tmp = (test_trans[test_trans['중분류'].isin(middle_keywords)].groupby('ID')['구매가격'].agg('sum')).reset_index().rename(columns = {'구매가격':'중분류_주요물품_구매액_합계'})
test_ft = test_ft.merge(tmp,how='left',on = 'ID')

train_ft.shape,test_ft.shape

((14940, 21), (12225, 20))

In [ ]:
train_ft = train_ft.fillna(0)

In [ ]:
test_ft = test_ft.fillna(0)

In [ ]:
key_words = ['공산품파트','공산품','가정용품','케주얼_구두_아동','생식품파트','생식품','남성정장스포츠','골프_유니캐쥬얼','여성정장','여성의류파트','로얄부틱']

tmp = (train_trans[train_trans['대분류'].isin(key_words)].groupby('ID')['구매가격'].agg('sum')).reset_index().rename(columns = {'구매가격':'대분류_주요물품_구매액_합계'})
train_ft = train_ft.merge(tmp,how='left',on = 'ID')
train_ft.fillna(0,inplace = True)

tmp = (test_trans[test_trans['대분류'].isin(key_words)].groupby('ID')['구매가격'].agg('sum')).reset_index().rename(columns = {'구매가격':'대분류_주요물품_구매액_합계'})
test_ft = test_ft.merge(tmp,how='left',on = 'ID')
test_ft.fillna(0,inplace = True)

In [ ]:
tmp = train_trans.groupby('ID')['대분류'].agg(lambda x : np.mean(x.isin(middle_keywords))).reset_index().rename(columns = {'중분류':'중분류_구매_비율'})
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')['대분류'].agg(lambda x : np.mean(x.isin(middle_keywords))).reset_index().rename(columns = {'중분류':'중분류_구매_비율'})
test_ft = test_ft.merge(tmp, how='left')

In [ ]:
q1, q3 = train_trans["구매가격"].quantile([0.25,0.75])
iqr = q3 - q1
w = 1.5
box_min = q1 - iqr * w
box_max = q3 + iqr * w
box_min, box_max

def boxplot(x):
  if box_min < x < q1:
    return 0
  elif q1 < x < q3:
    return 1
  elif q3 < x < box_max:
    return 2
  else:
    return 3



In [ ]:
q1, q3 = train_trans.groupby('ID')['구매가격'].agg('sum').quantile([0.25,0.75])
iqr = q3 - q1
w = 1.5
box_min = q1 - iqr * w
box_max = q3 + iqr * w
box_min, box_max

def boxplot(x):
  if box_min < x < q1:
    return 0
  elif q1 < x < q3:
    return 1
  elif q3 < x < box_max:
    return 2
  elif (0 < x < box_min)|(x > box_max):
    return 3
  elif x < 0:
    return 4

In [ ]:
train_trans['구매가격_분리'] = train_trans['구매가격'].apply(boxplot)

In [ ]:
test_trans['구매가격_분리'] = test_trans['구매가격'].apply(boxplot)

In [ ]:
tmp = train_trans.groupby('ID')['구매가격_분리'].agg(lambda x : np.mean(x == 1)).reset_index().rename(columns = {'구매가격_분리':'IQR비율'})
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')['구매가격_분리'].agg(lambda x : np.mean(x == 1)).reset_index().rename(columns = {'구매가격_분리':'IQR비율'})
test_ft = test_ft.merge(tmp, how='left')

In [ ]:
agg_list = [
        ('환불건수', lambda x: ( x < 0 ).sum() ),
    ]

tmp = train_trans.groupby(['ID'])["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft['4회이상_환불여부'] = np.where(train_ft['환불건수'] >= 4, 1, 0)

tmp = test_trans.groupby(['ID'])["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft['4회이상_환불여부'] = np.where(test_ft['환불건수'] >= 4, 1, 0)

In [ ]:
agg_list = [
        ('총구매액',lambda x: x[x > 0].sum() ),
        ('구매건수', lambda x: x[x > 0].count() ),
        ('평균구매액', lambda x: x[x > 0].mean()),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ]

tmp = train_trans.groupby(['ID'])["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft['구매금액표준편차'].fillna(0,inplace=True)
train_ft['확정구매']=train_ft['구매건수']-train_ft['환불건수']

tmp = test_trans.groupby(['ID'])["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft['구매금액표준편차'].fillna(0,inplace=True)
test_ft['확정구매']=test_ft['구매건수']-test_ft['환불건수']

In [ ]:
prod_list1 = ['생식품','생식품파트','공산품파트','공산품','잡화파트','가정용품파트','가정용품']
prod_list2 = ['아동_스포츠','아동_문화','케주얼_구두_아동','아동']
prod_list3 = ['여성캐쥬얼','패션잡화','여성정장','여성의류파트','여성캐주얼']
prod_list4 = ['남성정장스포츠','남성의류']
prod_list5 = ['스포츠캐쥬얼','영어덜트캐쥬얼','골프_유니캐쥬얼','잡화']
prod_list6 = ['로얄부띠끄','로얄부틱','명품잡화']
prod_list7 = ['영라이브','영플라자']

agg_list = [
    ('생활용품비율',lambda x : x.isin(prod_list1).mean() ),
    ('아동용품비율',lambda x : x.isin(prod_list2).mean() ),
    ('여성용품비율',lambda x : x.isin(prod_list3).mean() ),
    ('남성용품비율',lambda x : x.isin(prod_list4).mean() ),
    ('남녀공용용품비율',lambda x : x.isin(prod_list5).mean() ),
    ('명품비율',lambda x : x.isin(prod_list6).mean() ),
    ('기타비율',lambda x : x.isin(prod_list7).mean() )
]
tmp = train_trans.groupby('ID')["대분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft['생활_아동_남녀공용']=train_ft['생활용품비율']*train_ft['아동용품비율']*train_ft['남녀공용용품비율']

tmp = test_trans.groupby('ID')["대분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft['생활_아동_남녀공용']=test_ft['생활용품비율']*test_ft['아동용품비율']*test_ft['남녀공용용품비율']
train_ft.shape,test_ft.shape

((14940, 42), (12225, 41))

In [ ]:
# 나중에 성능 떨어진다고 생각하면 얘 버린다!
tmp = train_trans[train_trans['구매가격_분리'] == 1].groupby('ID')['구매가격'].agg('sum').reset_index().rename(columns = {'구매가격':'IQR_합계'})
train_ft = train_ft.merge(tmp,how='left',on = 'ID')

tmp = test_trans[test_trans['구매가격_분리'] == 1].groupby('ID')['구매가격'].agg('sum').reset_index().rename(columns = {'구매가격':'IQR_합계'})
test_ft = test_ft.merge(tmp,how='left',on = 'ID')
train_ft.shape,test_ft.shape

((14940, 43), (12225, 42))

In [ ]:
#이거 상당히 중요한 피처임
#뽑은 것들 중 젤 잘 뽑아낸 것임
agg_worktime_visit = [
    ('근무시간_방문비율' , lambda x : np.mean((x.dt.hour >= 9) & (x.dt.hour <= 17)))
]

tmp = train_trans.groupby('ID')['구매일시'].agg(agg_worktime_visit).reset_index()

train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')['구매일시'].agg(agg_worktime_visit).reset_index()

test_ft = test_ft.merge(tmp, how='left')

In [ ]:
def search_season(data):
    tmp=data.groupby('ID')[['봄_구매비율','여름_구매비율','가을_구매비율','겨울_구매비율']].max().idxmax(axis=1).apply(lambda x: x.split('_')[0])
    tmp=tmp.reset_index().rename(columns={0:'주구매계절'})
#     count_dict = tmp["주구매계절"].value_counts().to_dict()
#     tmp["주구매계절"]=tmp["주구매계절"].map(count_dict)
    return tmp

train_ft=train_ft.merge(search_season(train_ft), how='left')

In [ ]:
def search_season(data):
    tmp=data.groupby('ID')[['봄_구매비율','여름_구매비율','가을_구매비율','겨울_구매비율']].max().idxmax(axis=1).apply(lambda x: x.split('_')[0])
    tmp=tmp.reset_index().rename(columns={0:'주구매계절'})
#     count_dict = tmp["주구매계절"].value_counts().to_dict()
#     tmp["주구매계절"]=tmp["주구매계절"].map(count_dict)
    return tmp

test_ft=test_ft.merge(search_season(test_ft), how='left')

In [ ]:
train_ft.shape,test_ft.shape

((14940, 45), (12225, 44))

In [ ]:
encoder=OneHotEncoder(handle_unknown='ignore')

col=['주구매계절']

encoded_train=encoder.fit_transform(train_ft[col])

encoded_train_df = pd.DataFrame(encoded_train.toarray(), columns=encoder.get_feature_names_out(col))
train_ft = pd.concat([train_ft,encoded_train_df],axis=1)
train_ft.drop('주구매계절',axis=1,inplace=True)

In [ ]:
col=['주구매계절']

encoded_train=encoder.transform(test_ft[col])

encoded_train_df = pd.DataFrame(encoded_train.toarray(), columns=encoder.get_feature_names_out(col))
test_ft = pd.concat([test_ft,encoded_train_df],axis=1)
test_ft.drop('주구매계절',axis=1,inplace=True)

In [ ]:
train_trans["구매날짜"] = train_trans["구매일시"].dt.date
tmp = train_trans.groupby(["ID","구매날짜"])["구매일시"].agg(lambda x : (x.max() - x.min()).total_seconds() ).reset_index()

agg_list = [
    ("체류시간_max","max"),
    ("체류시간_min","min"),
    ("체류시간_std","std"),
    ("체류시간_mean","mean"),
    ("체류시간_sum","sum")
]
tmp2 = tmp.groupby("ID")["구매일시"].agg(agg_list).reset_index()
train_ft = pd.merge(train_ft, tmp2, how="left", on="ID")
train_ft['체류시간_std'].fillna(0,inplace=True)

test_trans["구매날짜"] = test_trans["구매일시"].dt.date
tmp = test_trans.groupby(["ID","구매날짜"])["구매일시"].agg(lambda x : (x.max() - x.min()).total_seconds() ).reset_index()

agg_list = [
    ("체류시간_max","max"),
    ("체류시간_min","min"),
    ("체류시간_std","std"),
    ("체류시간_mean","mean"),
    ("체류시간_sum","sum")
]
tmp2 = tmp.groupby("ID")["구매일시"].agg(agg_list).reset_index()
test_ft = pd.merge(test_ft, tmp2, how="left", on="ID")
test_ft['체류시간_std'].fillna(0,inplace=True)

In [ ]:
train_ft.fillna(0,inplace = True)
test_ft.fillna(0,inplace = True)

In [ ]:
agg_list = [
    ('구매금액의 중간값',lambda x : x[x>0].median()),
    ]

tmp = train_trans.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_trans.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape,test_ft.shape

((14940, 54), (12225, 53))

In [ ]:
tmp = train_trans.groupby('ID')['대분류'].agg(lambda x : np.mean(x.isin(['영플라자','영캐릭터']))>0.0508).astype(int).reset_index().rename(columns = {'대분류':'영플라자_영캐리턱_구매력'})
train_ft = train_ft.merge(tmp,how = 'left',on = 'ID')

tmp = test_trans.groupby('ID')['대분류'].agg(lambda x : np.mean(x.isin(['영플라자','영캐릭터']))>0.0508).astype(int).reset_index().rename(columns = {'대분류':'영플라자_영캐리턱_구매력'})
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape,test_ft.shape

((14940, 55), (12225, 54))

In [ ]:
tmp = train_trans.groupby('ID')['대분류'].agg(lambda x : np.mean(x.isin(['영플라자','영캐릭터']))).reset_index().rename(columns = {'대분류':'영플라자_영캐리턱_구매_비율'})
train_ft = train_ft.merge(tmp,how = 'left',on = 'ID')

tmp = test_trans.groupby('ID')['대분류'].agg(lambda x : np.mean(x.isin(['영플라자','영캐릭터']))).reset_index().rename(columns = {'대분류':'영플라자_영캐리턱_구매_비율'})
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape,test_ft.shape

((14940, 56), (12225, 55))

In [ ]:
tmp = train_trans.pivot_table(index = ['ID'],
                                columns = ['중분류'],
                                values = ['구매가격'],
                                aggfunc = np.sum)
tmp = tmp.reset_index()
tmp.fillna(0,inplace = True)
tmp.columns = ['_'.join(t) for t in tmp.columns]
tmp.rename(columns = {'ID_':'ID'},inplace = True)


train_ft = pd.merge(train_ft,tmp,how = 'left',on = 'ID')

tmp = test_trans.pivot_table(index = ['ID'],
                                columns = ['중분류'],
                                values = ['구매가격'],
                                aggfunc = np.sum)

tmp = tmp.reset_index()
tmp.fillna(0,inplace = True)
tmp.columns = ['_'.join(t) for t in tmp.columns]
tmp.rename(columns = {'ID_':'ID'},inplace = True)

test_ft = pd.merge(test_ft,tmp,how = 'left',on = 'ID')

train_ft.drop(columns = ['구매가격_특판'],inplace = True)
train_ft.shape

train_ft.shape,test_ft.shape

((14940, 359), (12225, 357))

In [ ]:
tmp = train_trans.pivot_table(index = ['ID'],
                                columns = ['대분류'],
                                values = ['구매가격'],
                                aggfunc = np.sum)
tmp = tmp.reset_index()
tmp.fillna(0,inplace = True)
tmp.columns = ['_'.join(t) for t in tmp.columns]
tmp.rename(columns = {'ID_':'ID'},inplace = True)
tmp

train_ft = pd.merge(train_ft,tmp,how = 'left',on = 'ID')

tmp = test_trans.pivot_table(index = ['ID'],
                                columns = ['대분류'],
                                values = ['구매가격'],
                                aggfunc = np.sum)
tmp = tmp.reset_index()
tmp.fillna(0,inplace = True)
tmp.columns = ['_'.join(t) for t in tmp.columns]
tmp.rename(columns = {'ID_':'ID'},inplace = True)
tmp

test_ft = pd.merge(test_ft,tmp,how = 'left',on = 'ID')

train_ft.shape,test_ft.shape

((14940, 386), (12225, 385))

In [ ]:
## 추가하지 못한 특성들 ##

# train_trans['구매시간대'] = train_trans['구매일시'].dt.hour
# test_trans['구매시간대'] = test_trans['구매일시'].dt.hour

# conditions = [
    # (train_trans['구매시간대'] < 9) | (train_trans['구매시간대'] > 17),
    # ((train_trans['구매시간대'] >= 9) & (train_trans['구매시간대'] <= 17)),
    # ]
# 구매시간대_분류 = ['근무시간대x', '근무시간대']

# train_trans['구매시간대_분리'] = np.select(conditions, 구매시간대_분류, default='Other')


# tmp = train_trans.pivot_table(index = ['ID'],
                                # columns = ['지점코드','구매시간대_분리'],
                                # values = ['구매가격'],
                                # aggfunc = np.sum)

# tmp.columns = [col1 + "_" + col2 + '_' + col3 for col1,col2,col3 in tmp.columns]

# tmp = tmp.reset_index()

# tmp.fillna(0,inplace = True)

# train_ft = train_ft.merge(tmp, how='left')


# conditions = [
    # (test_trans['구매시간대'] < 9) | (test_trans['구매시간대'] > 17),
    # ((test_trans['구매시간대'] >= 9) & (test_trans['구매시간대'] <= 17)),
    # ]
# 구매시간대_분류 = ['근무시간대x', '근무시간대']

# test_trans['구매시간대_분리'] = np.select(conditions, 구매시간대_분류, default='Other')

# tmp = test_trans.pivot_table(index = ['ID'],
                                # columns = ['지점코드','구매시간대_분리'],
                                # values = ['구매가격'],
                                # aggfunc = np.sum)

# tmp.columns = [col1 + "_" + col2 + '_' + col3 for col1,col2,col3 in tmp.columns]

# tmp = tmp.reset_index()

# tmp.fillna(0,inplace = True)

# test_ft = test_ft.merge(tmp, how='left')

# train_ft.shape,test_ft.shape

((14940, 63), (12225, 62))

In [ ]:
## 추가하지 못한 특성들 ##
## 열의 개수를 맞춰야 했는데 시간이 부족해서 찾지 못함 ##

# 고객별로 방문한 지점의 시간대 따른 구매가격의 평균
# train_trans['구매시간대'] = train_trans['구매일시'].dt.hour
# test_trans['구매시간대'] = test_trans['구매일시'].dt.hour

# tmp = train_trans.pivot_table(index = ['ID'],
                        # columns = ['지점코드','구매시간대'],
                        # values = ['구매가격'],
                        # aggfunc = np.mean)

# tmp.fillna(0,inplace=True)

# tmp.columns = [col1 + "_" + col2 + '_' + str(col3) for col1,col2,col3 in tmp.columns]

# tmp = tmp.reset_index()

# train_ft = train_ft.merge(tmp,how = 'left',on = 'ID')

# tmp = test_trans.pivot_table(index = ['ID'],
                        # columns = ['지점코드','구매시간대'],
                        # values = ['구매가격'],
                        # aggfunc = np.mean)

# tmp.fillna(0,inplace=True)

# tmp.columns = [col1 + "_" + col2 + '_' + str(col3) for col1,col2,col3 in tmp.columns]

# tmp = tmp.reset_index()

# test_ft = test_ft.merge(tmp,how = 'left',on = 'ID')

# train_ft.shape,test_ft.shape

((14940, 108), (12225, 112))

In [ ]:
## 추가하지 못한 특성들 ##

# 고객별로 지점에 따라 구매한 브랜드 코드의 종류
# brandcode_num = train_trans.groupby(['ID','지점코드'])['브랜드코드'].agg('nunique').reset_index().rename(columns = {'브랜드코드':'브랜드코드_nunique'})

# tmp = brandcode_num.pivot_table(index = ['ID'],
                                # columns = ['지점코드'],
                                # values = ['브랜드코드_nunique'],
                                # aggfunc = np.sum) # np.meam을하든 np.sum을 하든 동일하게 고객별로 구매한 nunique의 수가 나온다.

# tmp.columns = ['_'.join(t) for t in tmp.columns]

# train_ft = train_ft.merge(tmp,how = 'left',on = 'ID')

# brandcode_num = test_trans.groupby(['ID','지점코드'])['브랜드코드'].agg('nunique').reset_index().rename(columns = {'브랜드코드':'브랜드코드_nunique'})

# tmp = brandcode_num.pivot_table(index = ['ID'],
                                # columns = ['지점코드'],
                                # values = ['브랜드코드_nunique'],
                                # aggfunc = np.sum) # np.meam을하든 np.sum을 하든 동일하게 고객별로 구매한 nunique의 수가 나온다.

# tmp.columns = ['_'.join(t) for t in tmp.columns]

# test_ft = test_ft.merge(tmp,how = 'left',on = 'ID')

# train_ft.shape,test_ft.shape

((14940, 67), (12225, 66))

In [ ]:
train_ft.fillna(0,inplace = True)
test_ft.fillna(0,inplace = True)

,ID,target,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,구매가격_여성캐주얼,구매가격_여성캐쥬얼,구매가격_영라이브,구매가격_영어덜트캐쥬얼,구매가격_영캐릭터,구매가격_영플라자,구매가격_잡화_y,구매가격_잡화파트,구매가격_케주얼_구두_아동,구매가격_패션잡화_y
0,train_0,1.0,11,23.090909,0.250000,0.050000,0.250000,0.400000,0.300000,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,320000.0,35000.0,377500.0
1,train_1,1.0,21,16.714286,0.023810,0.357143,0.166667,0.357143,0.119048,3,...,1360500.0,0.0,0.0,0.0,689000.0,1376700.0,0.0,0.0,39000.0,710000.0
2,train_2,0.0,56,6.339286,0.210526,0.464912,0.140351,0.175439,0.219298,0,...,1003000.0,0.0,0.0,0.0,0.0,0.0,0.0,370000.0,0.0,399400.0
3,train_3,0.0,92,3.728261,0.189573,0.379147,0.180095,0.236967,0.203791,3,...,2392000.0,0.0,0.0,0.0,674000.0,396800.0,0.0,6261000.0,8548400.0,450000.0
4,train_4,0.0,27,11.555556,0.258065,0.112903,0.612903,0.209677,0.064516,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,526000.0,351600.0,0.0


In [ ]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.3 MB/s

In [ ]:
X = train_ft.drop(['ID', 'target'], axis=1)
y= train_ft['target']
Y = test_ft.drop(['ID'], axis=1)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

X_scaled = scaler.fit_transform(X)

Y_scaled = scaler.transform(Y)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
Y_scaled = pd.DataFrame(Y_scaled, columns=Y.columns)

In [ ]:
X_scaled['target']=train_ft['target']

In [ ]:
from pycaret.classification import *
# include=['lightgbm', 'xgboost', 'catboost', 'gbc', 'rf', 'et', 'ada', 'dt', 'knn', 'nb', 'lr']
# PyCaret 설정 초기화
clf = setup(data=X_scaled, target='target', session_id=123,
            use_gpu=True, # GPU 사용 활성화
            fold_strategy='stratifiedkfold', # 교차 검증 전략
            fold=5) # 10겹 교차 검증

# 모델 비교 (15개 모델 포함, 5겹 교차 검증 사용, ROC AUC로 정렬)
best_model = compare_models(n_select=6, include=['lightgbm', 'xgboost', 'catboost', 'gbc', 'rf', 'et', 'ada', 'dt', 'knn', 'nb', 'lr'],
                            cross_validation=True, sort='AUC')

# 최고의 모델 출력
print(best_model)
#lr	Logistic Regression	0.7365	0.7905	0.6024	0.6890	0.6426	0.4354	0.4380	0.2560
#lr	Logistic Regression	0.7356	0.7907	0.6012	0.6878	0.6414	0.4335	0.4361	0.2400
#lr	Logistic Regression	0.7343	0.7914	0.5995	0.6857	0.6396	0.4306	0.4332	0.2660
#gbc	Gradient Boosting Classifier	0.7360	0.7922	0.5995	0.6890	0.6411	0.4339	0.4365	20.5840

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(14940, 385)"
4,Transformed data shape,"(14940, 385)"
5,Transformed train set shape,"(10458, 385)"
6,Transformed test set shape,"(4482, 385)"
7,Numeric features,384
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7361,0.7969,0.5985,0.6898,0.6408,0.4338,0.4367,19.6640
lightgbm,Light Gradient Boosting Machine,0.7387,0.7966,0.6163,0.6873,0.6497,0.4423,0.4441,3.7840
rf,Random Forest Classifier,0.7257,0.7827,0.5389,0.6953,0.6071,0.4015,0.4093,4.7420
xgboost,Extreme Gradient Boosting,0.7276,0.7811,0.6060,0.6699,0.6362,0.4194,0.4208,6.1320
ada,Ada Boost Classifier,0.7248,0.7786,0.6046,0.6653,0.6333,0.4139,0.4153,4.3160
et,Extra Trees Classifier,0.7224,0.7758,0.5202,0.6976,0.5958,0.3912,0.4011,4.8320
lr,Logistic Regression,0.6735,0.7107,0.5268,0.5960,0.5590,0.3014,0.3030,9.7680
knn,K Neighbors Classifier,0.6201,0.6372,0.5221,0.5168,0.5193,0.2053,0.2054,1.8740
dt,Decision Tree Classifier,0.6438,0.6268,0.5474,0.5474,0.5472,0.2537,0.2539,1.4900
nb,Naive Bayes,0.5290,0.6023,0.8876,0.4499,0.5972,0.1573,0.2155,0.2920


Processing:   0%|          | 0/54 [00:00<?, ?it/s]

[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsa

In [ ]:
X_scaled.drop('target',axis=1,inplace=True)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV,KFold,cross_val_score
from sklearn.metrics import classification_report

cv=KFold(n_splits=5,random_state=42,shuffle=True)

lr = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


lr.fit(X_scaled, y)


# 최적의 모델로 예측 및 평가
cv_scores = cross_val_score(lr, X_scaled, y, cv=cv)

print("Cross-Validated Scores:", cv_scores)
print("Mean CV Score:", np.mean(cv_scores))


Cross-Validated Scores: [0.72523427 0.73795181 0.73728246 0.72824632 0.74029451]
Mean CV Score: 0.7338018741633199


In [ ]:
lr.predict_proba(Y_scaled)
pred=lr.predict_proba(Y_scaled)

submit = pd.DataFrame(pred[:,1], columns = ["target"])
submit['ID']=test_ft['ID']
submit=submit[['ID', 'target']]
submit
submit.to_csv("양태성12차.csv",index=False)

In [ ]:
X_scaled.to_excel(excel_writer = f'{DATA_PATH}X_scaled.xlsx')
Y_scaled.to_excel(excel_writer = f'{DATA_PATH}Y_scaled.xlsx')